In [1]:
import ast 
import pandas as pd
import seaborn as sns 

from datasets import load_dataset
import matplotlib.pyplot as plt 

dataset=load_dataset("lukebarousse/data_jobs")
df=dataset['train'].to_pandas()

df['job_posted_date']=pd.to_datetime(df['job_posted_date'])
df['job_posted_skill']=df['job_skills'].apply(lambda x:ast.literal_eval(x) if pd.notna(x) else x)

import ast


df['job_skills'] = df['job_skills'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


c:\Users\91982\anaconda3\envs\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_DA_india = df[(df['job_country']== 'India')& (df['job_title_short']=='Data Analyst')].copy()

In [8]:
df_DA_india=df_DA_india.dropna(subset=['salary_year_avg'])

df_DA_india_exploded=df_DA_india.explode('job_skills')
df_DA_india_exploded[['salary_year_avg', 'job_skills']].head(5)

,salary_year_avg,job_skills
11412,50400.0,python
11412,50400.0,sql
11412,50400.0,aws
11412,50400.0,azure
11412,50400.0,excel


In [10]:
df_DA_skills= df_DA_india_exploded.groupby('job_skills')['salary_year_avg'].agg(['count', 'median']).sort_values(by='count', ascending=False)

df_DA_skills =df_DA_skills.rename(columns={'count':'skill_count' , 'median':'median_salary'})